In [58]:
import os
import json 
import pandas as pd 
import traceback

In [59]:
from langchain.chat_models import ChatOpenAI

In [60]:
from langchain_openai import ChatOpenAI

In [66]:
KEY=os.getenv('OPENAI_API_API')

In [67]:
llm = ChatOpenAI(openai_api_key=KEY,model_name = "gpt-3.5-turbo")

In [68]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()  # Load .env file


True

In [70]:
KEY=os.getenv('OPENAI_API_KEY')

In [71]:
from dotenv import load_dotenv
load_dotenv()

True

In [72]:
print(KEY)

sk-proj-vGYb2GIjM86mxx0je9xwho8CJH6pqCd1diUnA6Ui
                ixT4eB2r21ggZJEGm5dTbTpZF5g0m2Eg5sT3BlbkFJQNsDa0k
                a8ti7DTWldwi412XUiuwxH76ZTepqIHC0acmQQM91DzA434Q
                5vf96A61E2eCtYT2QgA
